In [1]:
import pandas as pd

In [2]:
# Get data for all Test matches (March 8, 2004 - December 26, 2021) - compiled together from Cricsheet
data = pd.read_csv('all_matches.csv')

In [3]:
# Set qualifying team score info
qual_runs = 40
qual_wickets = 4

In [4]:
# Calculate running scores and wickets for each innings after each ball
def calc_running_score(innings):
    innings['score'] = (innings['runs_off_bat'] + innings['extras']).cumsum()
    wicket_condition = ~innings['wicket_type'].isnull() & ~innings['wicket_type'].isin(['retired hurt', 'retired not out'])
    innings['wickets'] = wicket_condition.cumsum()
    return innings

data = data.groupby(['match_id', 'innings']).apply(calc_running_score)

In [5]:
# Get all balls bowled to the new batsman coming in after the qualifying team score
def get_qual_balls(innings):
    index = innings.index[(innings['score'] <= qual_runs) & (innings['wickets'] == qual_wickets) & (~innings['player_dismissed'].isnull())]    
    if (len(index) == 0):
        return innings.head(0)
    else:
        innings = innings.truncate(index[0])
        striker, non_striker = innings.iloc[0]['striker'], innings.iloc[0]['non_striker']
        batsmen = innings['striker'].unique()
        new_batsman = [i for i in batsmen if i != striker and i != non_striker][0]
        return innings[innings['striker'] == new_batsman]

qual_data = data.groupby(['match_id', 'innings']).apply(get_qual_balls).reset_index(drop = True).sort_values(['start_date', 'innings', 'ball'])

In [21]:
# Calculate runs scored by each new batsman
new_batsmen_scores = qual_data.groupby(['match_id', 'start_date', 'innings', 'striker', 'batting_team', 'bowling_team'])['runs_off_bat'].sum().reset_index()

# Display the highest scores for a team
team = 'England'
table = new_batsmen_scores[new_batsmen_scores['batting_team'] == team].sort_values('runs_off_bat', ascending = False).reset_index(drop = True)
table.style.set_caption('Highest scores by #' + str(qual_wickets + 2) + ' batsmen for ' + team + ' after coming in at ' + str(qual_runs) + '/' + str(qual_wickets) + ' or less').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'black'),
        ('font-weight', 'bold'),
        ('font-size', '16px'),
        ('text-align', 'center')
    ]
}])

,match_id,start_date,innings,striker,batting_team,bowling_team,runs_off_bat
0,743939,2015-05-21,1,BA Stokes,England,New Zealand,92
1,743965,2015-07-16,2,BA Stokes,England,Australia,87
2,1140387,2018-11-23,3,JC Buttler,England,Sri Lanka,64
3,1031663,2017-09-07,2,BA Stokes,England,West Indies,60
4,226350,2005-11-20,4,A Flintoff,England,Pakistan,56
5,387573,2010-01-14,1,IR Bell,England,South Africa,35
6,1263462,2021-12-08,1,OJ Pope,England,Australia,35
7,300444,2008-03-22,1,PD Collingwood,England,New Zealand,30
8,352661,2009-02-04,3,A Flintoff,England,West Indies,24
9,1243385,2021-02-13,2,OJ Pope,England,India,22
